#Лабораторная работа №4
##по теме «Численное интегрирование»
##Васильев В. Вариант 8

##Задание: методом средних прямоугольников и методом трапеций найти интеграл: $$\int_1^2 x sin(x + 1) – cos(x – 5) dx$$
(c шагом 0.25)

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.integrate as intg

In [ ]:
def f(x):
  return x * np.sin(x + 1) - np.cos(x - 5)
  # return x

In [ ]:
# Given data
start = 1
end = 2
step_1 = 0.25
step_2 = step_1 / 2

In [ ]:
x_plot = np.linspace(start, end, 100)
y_plot = f(x_plot)

x_1 = np.arange(start, end + 0.1, step_1) # Для вычислений с обычным шагом
x_2 = np.arange(start, end + 0.1, step_2) # Для вычислений с половинным шагом
x_1

array([1.  , 1.25, 1.5 , 1.75, 2.  ])

In [ ]:
# Оценка погрешности (правило Рунге)
def Runge(Intg_1, Intg_2, method):
  if method not in ('rectangle', 'trapeze', 'Simpson'):
    return 'Неверный метод'
  elif method == 'Simpson':
    R = (Intg_2 - Intg_1) / 15
    Intg = Intg_2 + R
  else:
    R = (Intg_2 - Intg_1) / 3
    Intg = Intg_2 + R

  return (R, Intg)

##Вычислим интеграл по формуле средних прямоугольников: $$I = \sum_{i=1}^n f(\frac{x_{i-1}+x_i}{2}) h$$

In [ ]:
def rectangle_method(x, step):
  Intg = 0 # Значение интеграла

  # Узлы по методу прямоугольников (для проверки)
  x_dots = []
  y_dots = []

  for i in np.arange(1, len(x)):
    x_cur = (x[i-1] + x[i]) / 2 # Вычисление среднего соседних точек
    x_dots.append(x_cur)
    y_dots.append(f(x_cur))

    Intg += f(x_cur) * step

  return (Intg, x_dots, y_dots)

In [ ]:
# График функции и площадь, которую покрывает метод средних прямоугольников
def rectangle_method_plot(x_plot, y_plot, x, x_dots, y_dots):
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=x_plot, y=y_plot, name='График функции')) # Строим функцию

  # Строим средние прямоугольники
  for i in range(1, len(x)):

    fig.add_shape(type="rect",
                  xref="x", yref="y",
                  x0=x[i-1], y0=0,
                  x1=x[i], y1=y_dots[i-1],
                  opacity=0.2,
                  fillcolor="green",
                  line_color='green')

    # Добавляем пунктиром средние точки
    fig.add_trace(go.Scatter(x=[x_dots[i-1], x_dots[i-1]],
                             y=[0, y_dots[i-1]],
                             line=dict(color='#8e82d9', dash='dot'),
                             showlegend=False))

  # Добавление текста на прямоугольники
  fig.add_trace(go.Scatter(
      x=[1.5],
      y=[.5],
      text=["Площадь при помощи метода средних прямоугольников"],
      mode="text",
      name='Площадь',
      textfont=dict(size=20),
      showlegend=False
  ))



  fig.update_layout(title="Построение графика функции и площади по методу средних прямоугольников",
                  xaxis_title="Значение аргумента",
                  yaxis_title="Значение функции")

  return fig

In [ ]:
# Шаг h
Intg_1, x_dots, y_dots = rectangle_method(x_1, step_1)
Intg_1

1.702017214172867

In [ ]:
x_dots, x_1

([1.125, 1.375, 1.625, 1.875], array([1.  , 1.25, 1.5 , 1.75, 2.  ]))

In [ ]:
fig = rectangle_method_plot(x_plot, y_plot, x_1, x_dots, y_dots)
fig

In [ ]:
# Шаг h/2
Intg_2, x_dots, y_dots = rectangle_method(x_2, step_2)
Intg_2

1.6956874804491724

In [ ]:
fig = rectangle_method_plot(x_plot, y_plot, x_2, x_dots, y_dots)
fig

Погрешность по правилу Рунге: $R = \frac{I_{h/2} - I_h}{3}$

In [ ]:
R, Intg = Runge(Intg_1, Intg_2, 'rectangle')

R, Intg

(-0.0021099112412315732, 1.693577569207941)

##Вычислим интеграл по методу трапеций: $$I = \sum_{i=1}^n (f({x_{i-1})+f(x_i})) \frac{h}{2}$$

In [ ]:
def trapeze_method(x, step):
  Intg = 0

  # Узлы по методу прямоугольников (для проверки)
  x_dots = [x[0]]
  y_dots = [f(x[0])]

  for i in np.arange(1, len(x)):

    x_dots.append(x[i])
    y_dots.append(f(x[i]))

    Intg += (f(x[i]) + f(x[i-1])) * step / 2 # Накопление площадей трапеции

  return(Intg, x_dots, y_dots)

In [ ]:
# График функции и площадь, которую покрывает метод трапеций
def trapeze_method_plot(x_plot, y_plot, x_dots, y_dots, start, end):
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=x_plot,
                          y=y_plot,
                          name='График функции',
                          line=dict(color='#ff0000')))

  # Добавляем прямые (приближают функцию)
  fig.add_trace(go.Scatter(x=x_dots,
                          y=y_dots,
                          name='График приближения',
                          line=dict(color='#8e82d9')))
  # Ось Х
  fig.add_trace(go.Scatter(x=[start, end],
                          y=[0, 0],
                          line=dict(color='#8e82d9'),
                          name='Ось Х'))

  # Границы трапеции
  for i in np.arange(len(x_dots)):
    fig.add_trace(go.Scatter(x=[x_dots[i], x_dots[i]],
                          y=[0, y_dots[i]],
                          line=dict(color='#8e82d9', dash='dot'),
                          showlegend=False))

  fig.update_layout(title="Построение графика функции и площади по методу трапеций",
                  xaxis_title="Значение аргумента",
                  yaxis_title="Значение функции")

  return fig

In [ ]:
# Шаг h
Intg_1, x_dots, y_dots = trapeze_method(x_1, step_1)
Intg_1

1.6767347373182642

In [ ]:
fig = trapeze_method_plot(x_plot, y_plot, x_dots, y_dots, start, end)
fig

In [ ]:
# Шаг h/2
Intg_2, x_dots2, y_dots2 = trapeze_method(x_2, step_2)
Intg_2

1.6893759757455657

In [ ]:
fig = trapeze_method_plot(x_plot, y_plot, x_dots2, y_dots2, start, end)
fig

In [ ]:
R, Intg = Runge(Intg_1, Intg_2, 'trapeze')

R, Intg

(0.004213746142433861, 1.6935897218879996)

Рассчитаем интеграл с разными погрешностями и построим зависимость количества итераций от погрешностей (для метода трапеций)

In [ ]:
# Создаем списки для количества итераций и степеней погрешности
iter = []
exps = range(2, 11)

for exp in exps:
  E = 10**(-exp) # Погрешность
  n = 2 # Количество точек на отрезке
  i = 0 # Количество итераций для каждой погрешности
  R = 1 # Погрешность по правилу Рунге (сначала просто определили R)

  while abs(R) > E: # Проверяем погрешность

    # Наборы точек х с количеством точек равным n и n^2
    x1 = np.linspace(start, end, n)
    x2 = np.linspace(start, end, n*2)

    # Шаг для каждого из наборов
    h1 = x1[1] - x1[0]
    h2 = x2[1] - x2[0]

    # Вычисляем интегралы и погрешность методом двойного просчета (Рунге)
    I_1, x_dots, y_dots = trapeze_method(x1, h1)
    I_2, x_dots2, y_dots2 = trapeze_method(x2, h2)
    R, Intg = Runge(I_1, I_2, 'trapeze')

    n *= 2
    i += 1

  iter.append(i)


In [ ]:
iter

[2, 4, 5, 7, 9, 10, 12, 14, 15]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(exps),
                         y=iter,
                         mode='markers',
                         marker=dict(color='LightSkyBlue', size=15, line=dict(color='MediumPurple', width=3))))

fig.update_layout(title="Зависимость итераций от погрешности",
                  xaxis_title="Значение погрешности",
                  yaxis_title="Итерации")

##Вычислим интеграл по методу Симпсона: $$I = \sum_{i=1}^n \frac{h}{6} (f({x_{i-1}) + 4f(\frac{x_{i-1} + x_i}{2})+f(x_i}))$$

In [ ]:
def Simpson_method(x, step):

  Intg = 0
  coefficients = []
  x_dots = []
  for i in np.arange(1, len(x)):

    # Упростим написание точек
    x1, x2, x3 = x[i-1], (x[i-1] + x[i]) / 2, x[i]
    y1, y2, y3 = f(x1), f(x2), f(x3)

    # Определим коэффициенты параболы (для графика)
    a = (y3 - ((x3 * (y2 - y1) + x2*y1 - x1*y2) / (x2 - x1))) / (x3 * (x3 - x1 - x2) + x2*x1)
    b = (y2 - y1) / (x2 - x1) - a * (x1 + x2)
    c = (x2*y1 - x1*y2) / (x2 - x1) + a*x1*x2


    coefficients.append((a, b, c))
    x_dots.append((x1, x2, x3))
    Intg += (step / 6) * (f(x1) + 4 * f(x2) + f(x3))

  return (Intg, coefficients, x_dots)

In [ ]:
def Simpson_method_plot(x_plot, y_plot, coefficients, x_dots, start, end):
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=x_plot,
                           y=y_plot,
                           name='График функции',
                           line={'color': '#56c7a5'}))

  for i, coef in enumerate(coefficients):

    a, b, c = coef
    x1, x2, x3 = x_dots[i]
    x = np.linspace(x1, x3, 100) # Добавление х для построения параболы

    p2 = lambda x: a * x**2 + b * x + c # Определение многочлена 2-ой степени

    # Строим графики парабол
    fig.add_trace(go.Scatter(x=x,
                            y=p2(x),
                            name=f'Парабола {i+1}',
                            line={'color': '#8e82d9'}))

    for j, num in enumerate(x_dots[i]):

      if j % 2:
        color = '#8b8c27'
      else:
        color = '#8e82d9'

      fig.add_trace(go.Scatter(x=[num, num],
                                y=[0, p2(num)],
                                line=dict(color=color, dash='dot'),
                                showlegend=False))

  # Ось Х
  fig.add_trace(go.Scatter(x=[start, end],
                          y=[0, 0],
                          line=dict(color='#ab6c3f'),
                          name='Ось Х'))

  fig.update_layout(title="Построение графика функции и площади по методу Симпсона",
                  xaxis_title="Значение аргумента",
                  yaxis_title="Значение функции")

  return fig

In [ ]:
Intg_1, coefficients, x_dots = Simpson_method(x_1, step_1)
Intg_1

1.6935897218879994

In [ ]:
x_dots

[(1.0, 1.125, 1.25),
 (1.25, 1.375, 1.5),
 (1.5, 1.625, 1.75),
 (1.75, 1.875, 2.0)]

In [ ]:
fig = Simpson_method_plot(x_plot, y_plot, coefficients, x_dots, start, end)
fig

In [ ]:
Intg_2, coefficients, x_dots = Simpson_method(x_2, step_2)
Intg_2

1.6935836455479703

In [ ]:
fig = Simpson_method_plot(x_plot, y_plot, coefficients, x_dots, start, end)
fig

In [ ]:
R, Intg = Runge(Intg_1, Intg_2, 'Simpson')
R, Intg

(-4.0508933527583226e-07, 1.693583240458635)

Для более наглядного примера работы метода Симпсона можно расширишь интервал и увеличить шаг

In [ ]:
h = 10
start = -15
end = 15

x_plot = np.linspace(start, end, 1000)
y_plot = f(x_plot)

x1 = np.arange(start, end+h, h)
x2 = np.arange(start, end+h/2, h/2)


I1, coefs1, dots1 = Simpson_method(x1, h)
I2, coefs2, dots2 = Simpson_method(x2, h/2)

In [ ]:
x2

array([-15., -10.,  -5.,   0.,   5.,  10.,  15.])

In [ ]:
fig1 = Simpson_method_plot(x_plot, y_plot, coefs1, dots1, start, end)
# fig2 = Simpson_method_plot(x_plot, y_plot, coefs2, dots2, a, b)
fig1

###Графически поясним метод двойного просчета для метода симпсона

In [ ]:
for i, coef in enumerate(coefs2):

    a, b, c = coef
    x1, x2, x3 = dots2[i]
    x = np.linspace(x1, x3, 100) # Добавление х для построения параболы

    p2 = lambda x: a * x**2 + b * x + c # Определение многочлена 2-ой степени

    # Строим графики парабол
    fig1.add_trace(go.Scatter(x=x,
                              y=p2(x),
                              name=f'Парабола {i+1}',
                              line={'color': '#d90d87'}))

    for j, num in enumerate(dots2[i]):

      if j % 2:
        color = '#d9580d'
      else:
        color = '#d50dd9'

      fig1.add_trace(go.Scatter(x=[num, num],
                                y=[0, p2(num)],
                                line=dict(color=color, dash='dot'),
                                showlegend=False))

fig1

In [ ]:
I1, I2

(-37.42983584805651, 17.478218707715467)

In [ ]:
# Значение интеграла при помощи библиотечной функции
# y_ = f(x_)
# intg.simpson(y_, x_)